In [2]:
#import necessary modules
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

In [3]:
#define global variables
ID = 'id'
TARGET = 'loss'
SEED = 0
N_ESTIMATORS = 1000

In [4]:
#read data
train = pd.read_csv("./input/train.csv")
test = pd.read_csv("./input/test.csv")
print(train.shape)
print(test.shape)

(188318, 132)
(125546, 131)


In [32]:
#split data into train and test data
N_TRAIN = 141238
train = dataset[:N_TRAIN]
test = dataset[N_TRAIN:]
print(train.shape)
print(test.shape)

(141238, 132)
(47080, 132)


In [33]:
#train = pd.read_csv(PATH_OF_TRAIN_DATASET)
#test = pd.read_csv(PATH_OF_TEST_DATASET)
#N_TRAIN = train.shape[0]

In [34]:
#Data preprocessing - One-Hot Encoding
data = pd.concat((train,test), axis=0,ignore_index=True)
loss_all = data[TARGET]
id_all = data[ID]
loss_all_backup = loss_all

del data[TARGET]
del data[ID]

data = pd.get_dummies(data)
print(data.shape)

(188318, 1153)


In [35]:
#Process data for model
x_train = np.array(data.iloc[:N_TRAIN,:])
y_train = np.log(loss_all[:N_TRAIN].ravel())

x_test = np.array(data.iloc[N_TRAIN:,:])
y_test = np.log(loss_all[N_TRAIN:].ravel())

print(x_train.shape)
print(x_test.shape)

(141238, 1153)
(47080, 1153)


In [36]:
#split loss and id data
loss_all = loss_all.to_frame()
id_all = id_all.to_frame()
loss_train = loss_all[:N_TRAIN]
id_train = id_all[:N_TRAIN]

loss_test = loss_all[N_TRAIN:]
id_test = id_all[N_TRAIN:]

In [10]:
#Define ML model and train
model = XGBRegressor(n_estimators=N_ESTIMATORS,seed=SEED,max_depth=6,learning_rate=0.075,objective='reg:linear',colsample_bytree=0.7,subsample=0.7)
model.fit(x_train,y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.7,
       gamma=0, learning_rate=0.075, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.7)

In [11]:
#Calculate MAE
train_mae = mean_absolute_error(np.exp(y_train), np.exp(model.predict(x_train)))
print("Training Set MEA = %s" % train_mae)
test_mae = mean_absolute_error(np.exp(y_test), np.exp(model.predict(x_test)))
print("Testing Set MEA = %s" % test_mae)

Training Set MEA = 1003.48592105
Testing Set MEA = 1145.22727836


In [27]:
#Create file - test id + predicted loss
submission = pd.DataFrame()
pred = model.predict(x_test)
submission[ID] = id_test[ID]
submission[TARGET] = loss_test[TARGET]
submission['predicted loss'] = np.exp(pred)
submission.to_csv('Phaedrus_submission.csv', index=False)